In [18]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load Model

In [19]:
classifier_model = torch.load('best_G5_classifier.pt', map_location=torch.device('cpu'))
classifier_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Transform Images

In [20]:
# Read class names from txt
with open('class_names.txt', 'r') as f:
    class_names = f.read().splitlines()

# Dönüşümler
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

In [21]:
# Create VideoCapture object
cap = cv2.VideoCapture(0)

while True:
    # Get a frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip the image at horizontal axis
    frame = cv2.flip(frame, 1) 

    # Convert OpenCV image to array
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Transform image into suitable format for model
    image_tensor = transform(image).unsqueeze(0)

    # Guess the result
    with torch.no_grad():
        output = classifier_model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
        predicted_class = class_names[predicted.item()]
        confidence_percentage = confidence.item() * 100

    # Show what model guess on the image
    text = f'Predicted: {predicted_class} ({confidence_percentage:.2f}%)'
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (230, 230, 230), 2, cv2.LINE_AA)

    # Show the image
    cv2.imshow('Camera', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release camera and windows
cap.release()
cv2.destroyAllWindows()
